In [ ]:
# !pip install -U langchain-community
# !pip install faiss-cpu
# !pip install transformers accelerate bitsandbytes
# !pip install llama-cpp-python langchain-community


# Paso 1: Importar librerías

In [ ]:
import os
import re

from langchain.schema import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_core.language_models.fake import FakeListLLM
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain_community.llms import LlamaCpp
from langchain.document_loaders import PyPDFLoader, TextLoader

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline

import chardet
import re
import unicodedata

# Paso 2: Cargar archivo txt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Read the raw bytes of the file
with open("manual_usuario_mi_coto.txt", "rb") as file:
    raw_data = file.read()

# Detect the encoding
result = chardet.detect(raw_data)
encoding = result['encoding']
print(f"Detected encoding: {encoding}")

Detected encoding: MacRoman


# Paso 3: Ejecutar limpieza de texto

In [ ]:
def clean_text(text: str) -> str:
    # 1) Eliminar cabeceras/pies de página comunes
    text = re.sub(r"Página\s*\d+\s*/\s*\d+", " ", text, flags=re.IGNORECASE)
    text = re.sub(r"Manual de Usuario|Funcionalidad MiCoto", " ", text)

    # 2) Deshacer guiones al final de renglón: “co-\nmentario” → “comentario”
    text = re.sub(r"(\w)-\s*\n\s*(\w)", r"\1\2", text)

    # 3) Unir saltos de línea dentro del mismo párrafo
    text = re.sub(r"(?<!\n)\n(?!\n)", " ", text)

    # 4) Quitar URLs, correos y teléfonos
    text = re.sub(r"\bhttps?://\S+\b", " ", text)
    text = re.sub(r"\b[\w\.-]+@[\w\.-]+\.\w{2,}\b", " ", text)
    text = re.sub(r"\b\d{2,4}[-\s]?\d{2,4}[-\s]?\d{2,4}\b", " ", text)

    # 5) Unificar comillas y guiones largos
    text = text.replace("“", '"').replace("”", '"').replace("—", "-")
    # 6) Colapsar múltiples espacios y líneas en blanco
    text = re.sub(r"[ \t]{2,}", " ", text)
    text = re.sub(r"\n{3,}", "\n\n", text)

    return text.strip()

In [ ]:
# Read raw file and save clean content

with open("manual_usuario_mi_coto.txt", encoding = "MacRoman") as file:
    raw_data = file.read()
    content = clean_text(raw_data)

with open("manual_usuario_mi_coto_clean.txt", "w", encoding="MacRoman") as file:
    file.write(content)


# Paso 4: generar objeto TextLoader con archivo de texto preprocesado

In [ ]:
loader = TextLoader("manual_usuario_mi_coto_clean.txt", encoding="MacRoman")
documents = loader.load()

# Paso 5: Separar el texto en textos más cortos

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=100)
docs = text_splitter.split_documents(documents)


# Paso 6: Cargar modelo de embeddings

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

<ipython-input-9-83794808db26>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or da

# Paso 7: Crear indice con FAISS

In [ ]:
db = FAISS.from_documents(docs, embeddings)

# Paso 8: Guardar indice localmente

In [ ]:
db.save_local("micoto_faiss_index")

# Paso 9: Cargar indice para interacción

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# utilizar el mismo modelo empleado para indexar
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# cargar el archivo indice guardado de FAISS

db = FAISS.load_local("micoto_faiss_index", embeddings, allow_dangerous_deserialization=True)


# Paso 10: Comenzar a generar queries desde FAISS index

In [ ]:
# define k value

k_parameter = 3

In [ ]:
queries = [
    "¿Cómo cambio mi contraseña?",
    "¿Dónde puedo actualizar mi teléfono?",
    "¿Dónde se sube la constancia fiscal?",
    "¿Dónde veo mi saldo o mis adeudos?",
    "¿Cómo informo que ya pagué?",
    "¿Dónde agrego mis datos fiscales?",
    "¿Cómo aparto el salón de eventos?",
    "¿Cómo puedo cancelar una reservación?",
    "Envié un mensaje y no me han respondido",
    "¿Dónde veo lo que escribió el administrador?",
    "¿Dónde están las actas de la asamblea?",
    "¿Dónde está el reglamento del condominio?",
]


In [ ]:
for i, query in enumerate(queries, 1):
    print(f"\nPregunta {i}: {query}")
    results = db.similarity_search(query, k=k_parameter)
    for doc in results:
        print(doc.page_content)



Pregunta 1: ¿Cómo cambio mi contraseña?
tú tienes acceso a verla dentro de tu perfil en el portal y con la protección que MiCoto ofrece a sus usuarios.
SECCIÓN DE REPORTES: Aquí encontrarás diferentes tipos de reportes: 1. Por Casa: Vista general del estatus (al día, debe, atrasado). Se puede buscar por casa o aplicar filtros.Al seleccionar una casa, se muestran los adeudos con detalle. El administrador puede marcar como pagado, atrasado, modificar monto, hacer convenio o cancelar adeudo. 2. Mensuales: Vistazo a ingresos y egresos del mes. Se puede filtrar por tipo, estatus, buscar por casa o seleccionar mes/año. 3. 3. Anuales: Resumen por mes de pendientes, ingresos, egresos y balances anuales.
1. Paso 1 – ingresa al sitio www. micoto.mx : Ya sea que ingreses directamente desde nuestra app (disponible en Google play y appstore) o mediante a nuestra plataforma web puedes tener el 100% de la funcionalidad de esta herramienta. Nos encuentras como MiCoto.mx Dentro de nuestra plataforma w

# Paso 11: Probar rag

In [ ]:
# Usar un modelo falso para propósitos de prueba
test_responses = [
    "Puedes cambiar tu contraseña desde el perfil de usuario.",
    "Actualiza tu teléfono desde la sección de configuración de la cuenta.",
    "Sube la constancia fiscal en el apartado de Documentos fiscales.",
    "Puedes ver tu saldo y adeudos en la pestaña de Finanzas.",
    "Informa tu pago subiendo el comprobante en la sección correspondiente.",
    "Agrega tus datos fiscales desde el perfil.",
    "Aparta el salón desde la sección de Reservaciones.",
    "Cancela una reservación desde tu historial de reservaciones.",
    "Verifica si el mensaje fue recibido y revisa el buzón de salida.",
    "Lo que escribió el administrador aparece en el muro de avisos.",
    "Las actas de la asamblea están en el menú de Documentos.",
    "El reglamento del condominio se encuentra en la sección Legal."
]

llm = FakeListLLM(responses=test_responses)

prompt_template = PromptTemplate.from_template(
    "Responde en español basándote únicamente en el contexto proporcionado:\n\n{context}\n\nPregunta: {question}\nRespuesta:"
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 4}),
    chain_type_kwargs={"prompt": prompt_template}
)


In [ ]:
# generar preguntas

for q in queries:
    print(f"\nPregunta: {q}")
    print("Respuesta:", qa_chain.run(q))



Pregunta: ¿Cómo cambio mi contraseña?
Respuesta: Puedes cambiar tu contraseña desde el perfil de usuario.

Pregunta: ¿Dónde puedo actualizar mi teléfono?
Respuesta: Actualiza tu teléfono desde la sección de configuración de la cuenta.

Pregunta: ¿Dónde se sube la constancia fiscal?
Respuesta: Sube la constancia fiscal en el apartado de Documentos fiscales.

Pregunta: ¿Dónde veo mi saldo o mis adeudos?
Respuesta: Puedes ver tu saldo y adeudos en la pestaña de Finanzas.

Pregunta: ¿Cómo informo que ya pagué?


<ipython-input-17-9aec4c51eced>:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print("Respuesta:", qa_chain.run(q))


Respuesta: Informa tu pago subiendo el comprobante en la sección correspondiente.

Pregunta: ¿Dónde agrego mis datos fiscales?
Respuesta: Agrega tus datos fiscales desde el perfil.

Pregunta: ¿Cómo aparto el salón de eventos?
Respuesta: Aparta el salón desde la sección de Reservaciones.

Pregunta: ¿Cómo puedo cancelar una reservación?
Respuesta: Cancela una reservación desde tu historial de reservaciones.

Pregunta: Envié un mensaje y no me han respondido
Respuesta: Verifica si el mensaje fue recibido y revisa el buzón de salida.

Pregunta: ¿Dónde veo lo que escribió el administrador?
Respuesta: Lo que escribió el administrador aparece en el muro de avisos.

Pregunta: ¿Dónde están las actas de la asamblea?
Respuesta: Las actas de la asamblea están en el menú de Documentos.

Pregunta: ¿Dónde está el reglamento del condominio?
Respuesta: El reglamento del condominio se encuentra en la sección Legal.


# Paso 12: Implementación de RAG con llama

In [ ]:
# !wget https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf -O llama-2-7b-chat.Q4_K_M.gguf

In [ ]:
llm = LlamaCpp(
    model_path="llama-2-7b-chat.Q4_K_M.gguf",  # Path to your quantized model
    temperature=0.75,    # Slightly increased for more diversity
    max_tokens=512,      # Adjust to smaller tokens for faster generation
    top_p=0.85,          # Reduced to allow for faster responses
    top_k=50,            # Limit the candidate tokens considered
    n_ctx=2048,          # Decrease context window for faster processing
    verbose=False,       # Keep verbosity off for cleaner output
    n_threads=8,        # Increase threads for parallel processing (adjust based on your CPU)
)


llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_context: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized


In [ ]:
prompt_template = PromptTemplate.from_template(
    """Responde en español basándote únicamente en el contexto proporcionado:

{context}

Pregunta: {question}
Respuesta:"""
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 4}),
    chain_type_kwargs={"prompt": prompt_template}
)

In [ ]:
questions = [
    "¿Cómo cambio mi contraseña?",
    "¿Dónde puedo actualizar mi teléfono?",
    "¿Dónde se sube la constancia fiscal?",
    "¿Dónde veo mi saldo o mis adeudos?",
    "¿Cómo informo que ya pagué?",
    "¿Dónde agrego mis datos fiscales?",
    "¿Cómo aparto el salón de eventos?",
    "¿Cómo puedo cancelar una reservación?",
    "Envié un mensaje y no me han respondido",
    "¿Dónde veo lo que escribió el administrador?",
    "¿Dónde están las actas de la asamblea?",
    "¿Dónde está el reglamento del condominio?"
]

for q in questions:
    print(f"\nPregunta: {q}")
    print("Respuesta:", qa_chain.run(q))



Pregunta: ¿Cómo cambio mi contraseña?
Respuesta:  Para cambiar tu contraseña, haz clic en "Ingresar" en la parte superior derecha de la página y selecciona "Cambiar contraseña". Luego, ingresa tu antiguo password y el nuevo password que deseas utilizar. Asegúrate de verificar los campos obligatorios antes de guardar el cambio. Si tienes alguna duda o problema al cambiar tu contraseña, no dudes en contactarnos a través del menú "Soporte" en la parte superior izquierda de la página.

Pregunta: ¿Dónde puedo actualizar mi teléfono?
Respuesta:  En la sección de Notificaciones, podrás elegir si deseas recibir alertas por teléfono o correo. Si deseas actualizar tu teléfono, ingresa al menú de usuario y selecciona la opción "Notificaciones". Desde ahí, podrás activar o desactivar las notificaciones en función de tus necesidades.
Espero que esto te ayude a entender cómo funciona la plataforma de MiCoto. ¡Buena suerte!

Pregunta: ¿Dónde se sube la constancia fiscal?
Respuesta:  En la sección de